# Stable Diffusion Interactive Notebook 📓 🤖

A widgets-based interactive notebook that lets users generate AI images from prompts (Text2Image) using [Stable Diffusion](https://en.wikipedia.org/wiki/Stable_Diffusion).

<br/>

This notebook aims to be an alternative to WebUIs while offering a simple and lightweight GUI for anyone to get started with Stable Diffusion.

Uses [HuggingFace](https://huggingface.co/) Diffusers and [Jupyter widgets](https://github.com/jupyter-widgets/ipywidgets).

<br/>

Made with ❤️ by redromnon<br/>
[Learn more about this project](https://github.com/redromnon/stable-diffusion-interactive-notebook)

In [ ]:
#@title 👇 Installing dependencies { display-mode: "form" }
#@markdown ---
#@markdown Hit the run button to install all the necessary dependencies required to get things up and running.
#@markdown 
#@markdown Also make sure to select **GPU** as the runtime type:<br/>
#@markdown *Runtime->Change Runtime Type->Under Hardware accelerator, select GPU*
#@markdown 
#@markdown ---

!pip -q install torch torchvision diffusers transformers accelerate scipy safetensors xformers mediapy ipywidgets

In [ ]:
#@title 👇 Select SD model { form-width: "20%", display-mode: "form" }
#@markdown ---
#@markdown Run this cell to configure the pipeline for Stable Diffusion. This includes an option to select your preferred Stable Diffusion-based model from a given list.
#@markdown 
#@markdown Click on **Apply** to save the changes.
#@markdown 
#@markdown ---

from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler
from diffusers.models import AutoencoderKL
import torch
import ipywidgets as widgets
import importlib

#Pipe
pipe = None

#Models
select_model = widgets.Dropdown(
    options=[
        ("Stable Diffusion 2-1 Base" , "stabilityai/stable-diffusion-2-1-base"),
        ("Stable Diffusion 2-1" , "stabilityai/stable-diffusion-2-1"),
        ("Dreamlike Photoreal 2.0" , "dreamlike-art/dreamlike-photoreal-2.0"),
        ("OpenJourney v4" , "prompthero/openjourney-v4")
    ],
    description="Select Model:"
)

#Safety Checker
safety_check = widgets.Checkbox(
    value=True,
    description="Enable Safety Check",
    layout=widgets.Layout(margin="0px 0px 0px -85px")
)

#Output
out = widgets.Output()

#Apply Settings
apply_btn = widgets.Button(
    description="Apply",
    button_style="info"
)


#Run pipeline
def pipeline(p):

  global pipe
  
  out.clear_output()
  apply_btn.disabled = True

  with out:

    print("Running")
    
    pipe = StableDiffusionPipeline.from_pretrained(
      select_model.value, 
      scheduler=EulerAncestralDiscreteScheduler.from_pretrained(select_model.value, subfolder="scheduler"),
      torch_dtype=torch.float16,  
      vae=AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse", torch_dtype=torch.float16).to("cuda")
    ).to("cuda")

    if not safety_check.value:
      pipe.safety_checker = None

    pipe.enable_xformers_memory_efficient_attention()

  apply_btn.disabled = False


#Display
apply_btn.on_click(pipeline)

widgets.VBox(
    [
      widgets.HTML(value="<h2>Configure Pipeline</h2>"),
      select_model, safety_check, apply_btn, out
    ]
)


In [ ]:
#@title 👇 Generating Images { form-width: "20%", display-mode: "form" }
#@markdown ---
#@markdown Here comes the fun part! Running this cell presents you with a prompt text area and a wide variety of parameters for image generation.
#@markdown 
#@markdown Finally, hit **Generate** to create images!
#@markdown 
#@markdown ---
import ipywidgets as widgets, mediapy, random
import IPython.display


#PARAMETER WIDGETS 
width = "300px"

prompt = widgets.Textarea(
    value="",
    placeholder="Enter prompt",
    #description="Prompt:",
    rows=5,
    layout=widgets.Layout(width="600px")
)

neg_prompt = widgets.Textarea(
    value="",
    placeholder="Enter negative prompt",
    #description="Negative Prompt:",
    rows=5,
    layout=widgets.Layout(width="600px")
)

num_images = widgets.IntText(
    value=1,
    description="Images:",
    layout=widgets.Layout(width=width),
)

steps = widgets.IntText(
    value=30,
    description="Steps:",
    layout=widgets.Layout(width=width)
)

CFG = widgets.FloatText(
    value=7.5,
    description="CFG:",
    layout=widgets.Layout(width=width)
)

img_height = widgets.Dropdown(
    options=[('512px', 512), ('768px', 768)],
    value=512,
    description="Height:",
    layout=widgets.Layout(width=width)
)

img_width = widgets.Dropdown(
    options=[('512px', 512), ('768px', 768)],
    value=512,
    description="Width:",
    layout=widgets.Layout(width=width)
)

random_seed = widgets.IntText(
    value=-1,
    description="Seed:",
    layout=widgets.Layout(width=width),
    disabled=False
)

generate = widgets.Button(
    description="Generate",
    disabled=False,
    button_style="primary"
)

display_imgs = widgets.Output()


#RUN
def generate_img(i):

  #Clear output
  display_imgs.clear_output()

  generate.disabled = True

  with display_imgs: 

    print("Running")
    
    images = pipe(
      prompt.value,
      height = img_height.value,
      width = img_width.value,
      num_inference_steps = steps.value,
      guidance_scale = CFG.value,
      num_images_per_prompt = num_images.value,
      negative_prompt = neg_prompt.value,
      generator = torch.Generator("cuda").manual_seed(random_seed.value),
    ).images 
    mediapy.show_images(images)

  generate.disabled = False

#Display
generate.on_click(generate_img)

widgets.VBox(
    [
      widgets.AppLayout(
        header=widgets.HTML(
            value="<h2>Stable Diffusion</h2>",
        ),
        left_sidebar=widgets.VBox(
            [num_images, steps, CFG, img_height, img_width, random_seed]
        ),
        center=widgets.VBox(
            [prompt, neg_prompt, generate]
        ),
        right_sidebar=None,
        footer=None
      ),
      display_imgs
    ]
)